# Exercise: Chat with PDF (30 minutes)

Your goal in this exercise is to create a Gradio interface for a question-answering system.
Your application should:
- Use the query engine created above to answer questions about the uploaded PDF
- Display the question and answer in the UI. If using QueryEngine, use a question and answer format. If using ChatEngine, use a chat format.

If you need a challenge:
- Look at the LlamaIndex docs and add something new to the pipeline, such as [`HyDEQueryTransform`](https://docs.llamaindex.ai/en/stable/examples/query_transformations/HyDEQueryTransformDemo/)
- Use the `gr.File` component to allow the user to upload ANY pdf and ask question about it.

In [1]:
# !pip install gradio
!pip install llama-index

In [ ]:
# If we're in colab, use userdata to get the OPENAI_API_KEY
# In colab, you will need to restart the session after running this cell
# or you will get a PIL error when doing LLM inference.
import os
from rich import print
from pathlib import Path

try:
    from google.colab import userdata
    print("Colab detected - setting up environment")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    %pip install -qqqq llama-index \
        "openai==1.55.3" \
        "httpx==0.27.2" \
        llama-index-readers-web \
        thefuzz \
        gradio \
        chromadb \
        llama-index-embeddings-huggingface \
        llama-index-vector-stores-chroma \
        llama-index-retrievers-bm25 \
        llama-index-llms-gemini \
        docling \
        llama-index-readers-docling \
        llama-index-node-parser-docling \
        llama-index-utils-workflow \
        llama-index-tools-yahoo-finance \
        llama-index-tools-code-interpreter

except:
    print("Not in colab - using local environment variables.")
    from dotenv import load_dotenv
    load_dotenv("../.env")

In [ ]:

import gradio as gr
from tempfile import TemporaryDirectory

## Run this on terminal
#export OPENAI_API_KEY=XXXXX


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
# In the ingest_documents function:
def ingest_documents(file_obj):
    """
    Process an uploaded PDF file and create a query engine for it.

    Args:
        file_obj: Gradio file upload object containing the PDF

    Returns:
        tuple: (chat_engine, cleared_file_upload, filename)
    """
    if file_obj is None:
        return None, None, ""

    with TemporaryDirectory() as temp_dir:
        file_path = os.path.join(temp_dir, 'tmp.pdf')
        # Get the file bytes from the Gradio upload object
        file_bytes = open(file_obj, "rb").read()

        with open(file_path, "wb") as f:
            f.write(file_bytes)

        documents = SimpleDirectoryReader(temp_dir).load_data()
        llm = OpenAI(model="gpt-4o-mini")
        index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
        chat_engine = index.as_chat_engine(llm=llm, node_postprocessors=[
            SentenceTransformerRerank(top_n=5)
        ], retriever_top_k=20)

    return chat_engine, None, Path(file_obj.name).name, []

def chat(message, history, chat_engine):
    history.append(
        {
            "role": "user",
            "content": message
        }
    )
    response = chat_engine.chat(message)
    history.append({
        "role": "assistant",
        "content": response.response
    })
    return history, None

# In the Blocks interface:
with gr.Blocks() as demo:
    chat_engine_state = gr.State()  # Renamed for clarity
    gr.Markdown("## RAG Demo: Question answering with a PDF")
    with gr.Row():
        with gr.Column(scale=1):
            file_upload = gr.File(label="Upload a PDF file", file_types=[".pdf"], file_count="single")
            submit_button = gr.Button("Submit")
            pdf_name = gr.Textbox(label="You are asking about...")
        with gr.Column(scale=5):
            chatbot = gr.Chatbot(label="Chat with the uploaded PDF", type='messages')
            input = gr.Textbox(label="Enter a question", interactive=True)

    # Update to only store the chat_engine in the State
    submit_button.click(
        fn=ingest_documents,
        inputs=file_upload,
        outputs=[chat_engine_state, file_upload, pdf_name, chatbot]
    )
    input.submit(
        fn=chat,
        inputs=[input, chatbot, chat_engine_state],
        outputs=[chatbot, input]
    )

demo.launch(debug=False)